In [6]:
import numpy as np

# === 用户可配置路径 ===
tinyimages_bin_path = "./tinyimages/tiny_images.bin"  # TinyImages-80M 文件
subset_bin_path = "./tinyimages/tiny1M_clean_subset.bin"             # 输出 1M 子集文件
subset_indices_path = "./tinyimages/tiny1M_clean_indices.npy"        # 输出索引文件
subset_size = 1000000                                   # 1M 子集

# === 加载 cifar 重叠索引列表 ===
cifar_idxs_path = "../utils/80mn_cifar_idxs.txt"
with open(cifar_idxs_path, "r") as f:
    cifar_idxs = set(int(line.strip()) - 1 for line in f)

print(f"CIFAR 重叠样本数量: {len(cifar_idxs)}")

# === 生成可选索引列表（非 CIFAR 重叠样本） ===
total_images = 79302017
all_indices = np.arange(total_images, dtype=np.int64)
clean_indices = np.array([idx for idx in all_indices if idx not in cifar_idxs])
print(f"可用非重叠样本数量: {len(clean_indices)}")

# === 从 clean_indices 随机抽样 1M ===
subset_indices = np.random.choice(clean_indices, size=subset_size, replace=False)
np.save(subset_indices_path, subset_indices)
print(f"已保存子集索引: {subset_indices_path}")

# === 从全量 bin 文件中提取这些索引样本，写到新 bin 文件 ===
img_size = 32 * 32 * 3  # 每张图片大小（uint8）
with open(tinyimages_bin_path, "rb") as f_in, open(subset_bin_path, "wb") as f_out:
    for i, idx in enumerate(subset_indices):
        f_in.seek(idx * img_size)
        data = f_in.read(img_size)
        f_out.write(data)
        if i % 10000 == 0:
            print(f"已写入 {i} 张样本...")

print(f"✅ 已完成写入 1M 干净 TinyImages 子集到: {subset_bin_path}")



CIFAR 重叠样本数量: 132409
可用非重叠样本数量: 79169608
已保存子集索引: ./tinyimages/tiny1M_clean_indices.npy
已写入 0 张样本...
已写入 10000 张样本...
已写入 20000 张样本...
已写入 30000 张样本...
已写入 40000 张样本...
已写入 50000 张样本...
已写入 60000 张样本...
已写入 70000 张样本...
已写入 80000 张样本...
已写入 90000 张样本...
已写入 100000 张样本...
已写入 110000 张样本...
已写入 120000 张样本...
已写入 130000 张样本...
已写入 140000 张样本...
已写入 150000 张样本...
已写入 160000 张样本...
已写入 170000 张样本...
已写入 180000 张样本...
已写入 190000 张样本...
已写入 200000 张样本...
已写入 210000 张样本...
已写入 220000 张样本...
已写入 230000 张样本...
已写入 970000 张样本...
已写入 980000 张样本...
已写入 990000 张样本...
✅ 已完成写入 1M 干净 TinyImages 子集到: ./tinyimages/tiny1M_clean_subset.bin


In [3]:
!pwd

/workspace/aaii/DAL/data


In [4]:
!ls

LSUN	 dtd		      iSUN		places365  tinyimages
cifarpy  gen1MTinyImgs.ipynb  importdata.ipynb	svhn


In [ ]:
from utils.tinyimages_80mn_loader import TinyImages

# 用新生成的 1M 子集
tinyimages_bin_path = "./tiny_images/tiny1M_clean_subset.bin"

# 加载 Dataset（不排除 CIFAR，因为已经干净了）
dataset = TinyImages(transform=None, exclude_cifar=False)
dataset.__len__ = lambda: 1000000  # 因为 1M 子集，改掉 loader 里的 __len__

# 直接输出数量
print("图像总数:", len(dataset))